### 11. 서울시 범죄율 지도 시각화 

그럼 이제 우리 데이터를 가지고 지도 시각화를 해보죠. 그런데 우리는 서울시 지도 경계선을 구획해주는 json 파일을 만들지 못하죠? 그러면 남이 만든 파일을 가져다 쓰면 됩니다..^^(허락받구요ㅎㅎ) 누구든 사용할 수 있도록 해놓은 것이 있으니 우리는 이것을 쓰면 됩니다. 아래 파일 다운받기!

(github repository에도 올려 두었으니 다운로드해서 사용하시기 바랍니다.)

- 02. skorea_municipalities_geo_simple.json

*데이터 출처 : https://github.com/PinkWink/DataScience/tree/master/data

In [ ]:
# json 파일 로딩하기 
import json
geo_path = 'C:\\Users\\user\\Downloads\\02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

우선 살인 발생 건수부터 시각화해 볼게요. location에는 서울시 중심의 위도와 경보를 설정해주고요. 

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
              data = crime_anal_norm['살인'],
              columns = [crime_anal_norm.index, crime_anal_norm['살인']],
              fill_color = 'YlGnBu',
              key_on = 'feature.id')
map

C:\Users\user\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


흐음,,, 강남 3구가 살인 발생 건수에서 보면 안전하다고 보이지는 않네요.

그 다음에는 강간 발생 건수로 그려볼게요.

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='Stamen Toner')

map.choropleth(geo_data = geo_str,
              data = crime_anal_norm['강간,추행'],
              columns = [crime_anal_norm.index, crime_anal_norm['강간,추행']],
              fill_color = 'YlGnBu',
              key_on = 'feature.id')
map

응???? 강남이랑 서초구가 굉장히 짙죠... 흠... 클럽들이 많이 보여있어서 그런것 같은 느낌도 들구요...

범죄 자체로만 한번 볼게요.

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='Stamen Toner')

map.choropleth(geo_data = geo_str,
              data = crime_anal_norm['범죄'],
              columns = [crime_anal_norm.index, crime_anal_norm['범죄']],
              fill_color = 'YlGnBu',
              key_on = 'feature.id')
map

마찬가지로 강남 3구의 범죄 발생 건수가 작지 않네요. 

그러나 인구수 대비 범죄 발생율을 비교해 봅시다. 인구수가 많아서 범죄 건수 자체가 많을 수도 있으니까요.

In [ ]:
tmp_criminal = crime_anal_norm['범죄'] / crime_anal_norm['인구수'] * 1000000

map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='Stamen Toner')

map.choropleth(geo_data = geo_str,
              data = tmp_criminal,
              columns = [tmp_criminal.index, tmp_criminal],
              fill_color = 'YlGnBu',
              key_on = 'feature.id')
map

인구 대비로 본다 한들, 강남 3구가 안전도가 높다고 말하긴 어려울 것 같아요. 이렇게 기사를 검증해봤네요! 

### 12. 서울시 경찰서별 검거율과 구별 범죄 발생율 동시에 시각화 

경찰서별 검거율과 범죄 발생율을 동시에 표현해 보도록 하겠습니다. 우선 데이터에 경도와 위도를 칼럼으로 넣어주고요.

In [ ]:
crime_anal_raw['lat'] = station_lat
crime_anal_raw['lng'] = station_lng
crime_anal_raw.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,"강간,추행 발생","강간,추행 검거",절도 발생,절도 검거,폭력 발생,폭력 검거,구별,lat,lng
0,중부,1,1,3,4,113.0,76.0,943,495,997,830,중구,37.563617,126.989652
1,종로,9,6,4,4,115.0,93.0,602,284,964,792,종로구,37.575548,126.984747
2,남대문,1,1,2,2,73.0,52.0,699,368,579,494,중구,37.554758,126.973498
3,서대문,3,3,2,1,159.0,149.0,1077,524,1292,1029,서대문구,37.564744,126.966770
4,혜화,0,0,3,3,77.0,53.0,581,310,747,628,종로구,37.571843,126.998623


총 구별 검거를 구하기 위해 제일 큰 검거수로 정규화해서 표현해줍니다.

In [ ]:
col = ['살인 검거', '강도 검거', '강간,추행 검거', '절도 검거', '폭력 검거']
tmp = crime_anal_raw[col] / crime_anal_raw[col].max()
    
crime_anal_raw['검거'] = np.sum(tmp, axis=1)

crime_anal_raw.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,"강간,추행 발생","강간,추행 검거",절도 발생,절도 검거,폭력 발생,폭력 검거,구별,lat,lng,검거
0,중부,1,1,3,4,113.0,76.0,943,495,997,830,중구,37.563617,126.989652,1.281945
1,종로,9,6,4,4,115.0,93.0,602,284,964,792,종로구,37.575548,126.984747,1.536161
2,남대문,1,1,2,2,73.0,52.0,699,368,579,494,중구,37.554758,126.973498,0.854498
3,서대문,3,3,2,1,159.0,149.0,1077,524,1292,1029,서대문구,37.564744,126.966770,1.550739
4,혜화,0,0,3,3,77.0,53.0,581,310,747,628,종로구,37.571843,126.998623,0.838186


그럼 이제 folium으로 경찰서에다가 마크 표시를 해볼게요. 위도와 경도는 이때 쓰려고 앞의 google maps에서 가져온 겁니다.

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for n in crime_anal_raw.index:
    folium.Marker([crime_anal_raw['lat'][n], 
                   crime_anal_raw['lng'][n]]).add_to(map)
    
map

검거에다가 적당한 값(10)을 곱해서 원 넓이를 정할게요. 경찰서의 검거율을 원의 넓이로 표현하는게 되겠네요. 그럼 경찰서의 위치에서 넓은 원을 가지면 검거율이 높다고 볼 수 있겠죠? 

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for n in crime_anal_raw.index:
    folium.CircleMarker([crime_anal_raw['lat'][n], crime_anal_raw['lng'][n]],
                       radius = crime_anal_raw['검거'][n]*10,
                       color='#3186cc', fill_color='#3186cc').add_to(map)
    
map

그럼 이제 범죄 발생 건수도 함께 시각화해 보겠습니다. 

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

map.choropleth(geo_data = geo_str,
              data = crime_anal_norm['범죄'],
              columns = [crime_anal_norm.index, crime_anal_norm['범죄']],
              fill_color = 'YlGnBu',
              key_on = 'feature.id')

for n in crime_anal_raw.index:
    folium.CircleMarker([crime_anal_raw['lat'][n], crime_anal_raw['lng'][n]],
                       radius = crime_anal_raw['검거'][n]*10,
                       color='#3186cc', fill_color='#3186cc').add_to(map)
    
map

C:\Users\user\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


범죄가 더 많이 일어날수록 짙은 색이고, 경찰서의 방어력 정도를 동그라미 원이라 볼 수 있겠네요. 그런데,,, 강남 3구는 검거도 그렇고 방어율도 그렇고 그렇게 좋아보이지 않네요. 단순히 체감만 그런 것일까요. 저자가 얘기하듯, 강남 3구에 유흥업소가 많이 있기 때문에 이런 시각화가 나온 것 같은데요. 생각해볼 거리가 많은 것 같지만, 강남 3구의 실제 체감안전도는 검증해본 것 같습니다. 
